# ICU Length of Stay Prediction - MIMIC-III Pipeline

## 🎯 Objective
Predict ICU stay duration using PySpark ML on MIMIC-III dataset

## 📊 Data & Constraints
- **Sources**: 6 MIMIC-III tables (CHARTEVENTS, LABEVENTS, ICUSTAYS, etc.)
- **Filters**: 
        - Patient Age 18-80
        - LOS 0.1-15 days
        - Valid time sequences
- **Timeframe**: Vitals (first 24h), Labs (6h pre to 24h post ICU)


## 🌀 Big Data Processing

- **Storage**: We used Google Cloud Dataproc and Google Storage Buckets for MIMIC-III storage 
- **CHARTEVENTS**: Chart Events table has +330 million rows
- **Parquet**: Converted "CHARTEVENTS" and "LABEVENTS" tables to Parquet format for efficient storage and processing
- **Filtering**: We filtered immediately when loading to optimize CHARTEVENTS DataFrame

## 🔧 Features (39 total)
- **Demographics (2)**: Age, gender
- **Admission (8)**: Emergency/elective, timing, insurance
- **ICU Units (6)**: Care unit types, transfers
- **Vitals (11)**: HR, BP, RR, temp, SpO2 (avg/std)
- **Labs (8)**: Creatinine, glucose, electrolytes, blood counts
- **Diagnoses (4)**: Total count, sepsis, respiratory failure

## 🤖 Models & Results
- **Linear Regression**: 
- **Random Forest**: 

## ☁️ Infrastructure
- **GCP Dataproc**: 1x Master and 2x Workers, n2-standard-4  (12 vCPUs, 48GB RAM, 400GB Disk Storage)
- **Optimizations**: Smart sampling, aggressive filtering, 80/20 split





## Import Libraries

In [1]:
# Core PySpark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Machine Learning imports
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor, LinearRegression #, GBTRegressor
#from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator #, MulticlassClassificationEvaluator
#from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from pyspark.ml import Pipeline

from datetime import datetime, timedelta
import time

print("✅ All imports loaded successfully!")
print(f"⏰ Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All imports loaded successfully!
⏰ Notebook started at: 2025-06-02 15:27:31


## Setup Spark Session

In [2]:
spark = SparkSession.builder \
    .appName("Forecast-LOS") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    \
    .config("spark.executor.memory", "5g") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "2") \
    \
    .config("spark.driver.memory", "10g") \
    .config("spark.driver.cores", "3") \
    .config("spark.driver.maxResultSize", "2g") \
    \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "64MB") \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.adaptive.maxShuffledHashJoinLocalMapThreshold", "32MB") \
    \
    .config("spark.network.timeout", "600s") \
    .config("spark.sql.broadcastTimeout", "300s") \
    .config("spark.rpc.askTimeout", "300s") \
    \
    .config("spark.executor.heartbeatInterval", "20s") \
    .config("spark.dynamicAllocation.enabled", "false") \
    \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes", "128MB") \
    \
    .config("spark.executor.memoryOffHeap.enabled", "true") \
    .config("spark.executor.memoryOffHeap.size", "1g") \
    \
    .getOrCreate()
print("✅ Spark session created successfully!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔧 Application Name: {spark.sparkContext.appName}")
print(f"💾 Available cores: {spark.sparkContext.defaultParallelism}")
print(f"\n⏰ Spark session initialised at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/02 15:27:36 INFO SparkEnv: Registering MapOutputTracker
25/06/02 15:27:36 INFO SparkEnv: Registering BlockManagerMaster
25/06/02 15:27:36 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/06/02 15:27:36 INFO SparkEnv: Registering OutputCommitCoordinator


✅ Spark session created successfully!
📊 Spark Version: 3.5.3
🔧 Application Name: Forecast-LOS
💾 Available cores: 2

⏰ Spark session initialised at: 2025-06-02 15:27:43


# Load Data

Strategy: Pre-filter CHARTEVENTS to find ICU stays with required vital signs, then efficiently load all tables using broadcast joins and lookup tables.
Key Steps:

- Filter for ICU stays with ≥1 of 6 vital signs (HR, BP, RR, Temp, SpO2)
- Create lookup tables for ICUSTAY_ID, HADM_ID, SUBJECT_ID
- Load all tables with pre-filtering using broadcast joins
- Convert large files to "Parquet" for performance

Result: Memory-efficient loading of only relevant data with quality assurance that all ICU stays have vital signs measurements.

In [3]:
# Configuration flags
SAMPLE_ENABLE = False
SAMPLE_SIZE = 20000
MIMIC_PATH = "gs://dataproc-staging-europe-west2-851143487985-hir6gfre/mimic-data"



print("🏥 Loading MIMIC-III data...")

# Step 1: First, find ICUSTAY_IDs that have ALL required vital signs
print("📂 Loading CHARTEVENTS...")

try:
    chartevents_df = spark.read.parquet(f"{MIMIC_PATH}/CHARTEVENTS.parquet")
    print("✅ Loaded CHARTEVENTS from parquet")
except:
    print("📄 Converting CHARTEVENTS.csv.gz to parquet...")
    chartevents_csv = spark.read.option("header", "true").option("inferSchema", "false").csv(f"{MIMIC_PATH}/CHARTEVENTS.csv.gz")
    chartevents_csv.write.mode("overwrite").parquet(f"{MIMIC_PATH}/CHARTEVENTS.parquet")
    chartevents_df = spark.read.parquet(f"{MIMIC_PATH}/CHARTEVENTS.parquet")
    print("✅ Converted and loaded CHARTEVENTS")



# Step 2: Load ICUSTAYS 
print("\n📂 Loading and filtering ICUSTAYS...")
icustays_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/ICUSTAYS.csv.gz")



# Step 3: Apply sampling if enabled
if SAMPLE_ENABLE:
    print(f"🎯 Sampling {SAMPLE_SIZE} ICU stays...")
    icustays_df = icustays_df.limit(SAMPLE_SIZE)
    icustays_df.cache()
    actual_sample_size = icustays_df.count()
    print(f"✅ Final sample: {actual_sample_size} ICU stays")
else:
    icustays_df.cache()
    actual_sample_size = icustays_df.count()

    
    
# Step 4: Create efficient lookup tables
print("📋 Creating ID lookup tables...")
icu_lookup = icustays_df.select("ICUSTAY_ID").distinct().cache()
hadm_lookup = icustays_df.select("HADM_ID").distinct().cache()
subject_lookup = icustays_df.select("SUBJECT_ID").distinct().cache()

icu_lookup.count()  # Trigger caching
hadm_lookup.count()
subject_lookup.count()

# Step 5: Load other tables with optimized joins
print("📂 Loading PATIENTS table...")
patients_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/PATIENTS.csv.gz")
patients_df = patients_df.join(broadcast(subject_lookup), "SUBJECT_ID", "inner")

print("📂 Loading ADMISSIONS table...")
admissions_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/ADMISSIONS.csv.gz")
admissions_df = admissions_df.join(broadcast(hadm_lookup), "HADM_ID", "inner")

print("📂 Loading DIAGNOSES_ICD table...")
diagnoses_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/DIAGNOSES_ICD.csv.gz")
diagnoses_df = diagnoses_df.join(broadcast(hadm_lookup), "HADM_ID", "inner")

# Step 6: Load and filter CHARTEVENTS efficiently
print("📂 Loading CHARTEVENTS table... [FILTERING BY ICUSTAY_ID]")
chartevents_df = chartevents_df \
    .select("ICUSTAY_ID", "CHARTTIME", "ITEMID", "VALUE", "VALUEUOM", "VALUENUM") \
    .join(broadcast(icu_lookup), "ICUSTAY_ID", "inner")

# Step 7: Load LABEVENTS
print("📂 Loading LABEVENTS table... [FILTERING BY HADM_ID]")
try:
    labevents_df = spark.read.parquet(f"{MIMIC_PATH}/LABEVENTS.parquet")
except:
    print("📄 Converting LABEVENTS.csv.gz to parquet...")
    labevents_csv = spark.read.option("header", "true").option("inferSchema", "false").csv(f"{MIMIC_PATH}/LABEVENTS.csv.gz")
    labevents_csv.write.mode("overwrite").parquet(f"{MIMIC_PATH}/LABEVENTS.parquet")
    labevents_df = spark.read.parquet(f"{MIMIC_PATH}/LABEVENTS.parquet")

labevents_df = labevents_df.join(broadcast(hadm_lookup), "HADM_ID", "inner")

# Final summary
print("\n✅ Data loading complete!")
print(f"📊 ICUSTAYS: {icustays_df.count():,} rows")
print(f"📊 PATIENTS: {patients_df.count():,} rows") 
print(f"📊 ADMISSIONS: {admissions_df.count():,} rows")
print(f"📊 DIAGNOSES_ICD: {diagnoses_df.count():,} rows")
print(f"📊 CHARTEVENTS (filtered): {chartevents_df.count():,} rows")
print(f"📊 LABEVENTS (filtered): {labevents_df.count():,} rows")
print(f"\n⏰ Data loaded at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🏥 Loading MIMIC-III data...
📂 Loading CHARTEVENTS...


25/06/02 15:28:00 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/02 15:28:15 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/02 15:28:30 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/02 15:28:45 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/02 15:29:00 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/02 15:29:15 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere

📄 Converting CHARTEVENTS.csv.gz to parquet...


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe287c46690>>
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe287c46690>>
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe287c46690>>
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_th

# Features Engineering



## Extracting Data From ICUSTAYS

**Purpose**: Create comprehensive ICU dataset by joining ICU stays with patient demographics and admission details.

**Key Features**:
- **Target Variable**: ICU_LOS_DAYS (length of stay)
- **Demographics**: Age (18-80), gender, ethnicity
- **Clinical**: Care units, admission type/location, insurance
- **Outcomes**: Hospital/patient death flags
- **Identifiers**: ICUSTAY_ID, SUBJECT_ID, HADM_ID

**Age Filter**: Adults only (18-80 years) to exclude pediatric/very elderly edge cases.

**Result**: Clean base dataset ready for vital signs feature engineering.

In [ ]:
print("📊 Step 1: Creating base ICU dataset with patient demographics...")

base_icu_df = icustays_df.alias("icu") \
    .join(patients_df.alias("pat"), "SUBJECT_ID", "inner") \
    .join(admissions_df.alias("adm"), ["SUBJECT_ID", "HADM_ID"], "inner") \
    .select(
        # ICU stay identifiers
        col("icu.ICUSTAY_ID"),
        col("icu.SUBJECT_ID"), 
        col("icu.HADM_ID"),
        
        # Target variable - Length of Stay in ICU (days)
        col("icu.LOS").alias("ICU_LOS_DAYS"),
        
        # ICU characteristics
        col("icu.FIRST_CAREUNIT"),
        col("icu.LAST_CAREUNIT"), 
        col("icu.INTIME").alias("ICU_INTIME"),
        col("icu.OUTTIME").alias("ICU_OUTTIME"),
        
        # Patient demographics
        col("pat.GENDER"),
        col("pat.DOB"),
        col("pat.EXPIRE_FLAG").alias("PATIENT_DIED"),
        
        # Admission details
        col("adm.ADMITTIME"),
        col("adm.DISCHTIME"), 
        col("adm.ADMISSION_TYPE"),
        col("adm.ADMISSION_LOCATION"),
        col("adm.INSURANCE"),
        col("adm.ETHNICITY"),
        col("adm.HOSPITAL_EXPIRE_FLAG").alias("HOSPITAL_DEATH"),
        col("adm.DIAGNOSIS").alias("ADMISSION_DIAGNOSIS")
    )

# Calculate age at ICU admission
base_icu_df = base_icu_df.withColumn("AGE_AT_ICU_ADMISSION", \
                                     floor(datediff(col("ICU_INTIME"), col("DOB")) / 365.25)) \
                                     .filter(col("AGE_AT_ICU_ADMISSION").between(18,80))


print("✅ Created base ICU dataset!")


## Extracting Categorical Features

**Features Created**:
- **GENDER_BINARY**: Male = 1, Female = 0
- **CAME_FROM_ER**: Emergency admission = 1
- **HAS_INSURANCE**: Medicare = 1, other = 0
- **ADMISSION_TYPE_ENCODED**: Emergency=1, Elective=2, Urgent=3, Other=0
- **ETHNICITY_ENCODED**: White=1, Black=2, Hispanic=3, Asian=4, Other=5

**Result**: Categorical variables converted to numerical format for ML models.

In [ ]:
print("📊 Step 2: Engineering categorical features...")
base_icu_df = base_icu_df \
    .withColumn("GENDER_BINARY", when(col("GENDER") == "M", 1).otherwise(0)) \
    .withColumn("CAME_FROM_ER", when(col("ADMISSION_LOCATION").contains("EMERGENCY"), 1).otherwise(0)) \
    .withColumn("HAS_INSURANCE", when(col("INSURANCE") == "Medicare", 1).otherwise(0)) \
    .withColumn("ADMISSION_TYPE_ENCODED", 
                when(col("ADMISSION_TYPE") == "EMERGENCY", 1)
                .when(col("ADMISSION_TYPE") == "ELECTIVE", 2)
                .when(col("ADMISSION_TYPE") == "URGENT", 3)
                .otherwise(0)) \
    .withColumn("ETHNICITY_ENCODED",
                when(col("ETHNICITY").contains("WHITE"), 1) \
                .when(col("ETHNICITY").contains("BLACK"), 2) \
                .when(col("ETHNICITY").contains("HISPANIC"), 3) \
                .when(col("ETHNICITY").contains("ASIAN"), 4) \
                .otherwise(5))

print("✅ Base ICU Dataset - Categorical Features")


## Extracting ICU Unit Types

**Purpose**: Create categorical features for ICU unit types and transfers.

**Features Created**:
- **FIRST_UNIT_ENCODED**: Numerical encoding of ICU units
 - MICU (Medical) = 1
 - SICU (Surgical) = 2  
 - CSRU (Cardiac Surgery) = 3
 - CCU (Coronary Care) = 4
 - TSICU (Trauma Surgical) = 5
 - Other = 0
- **CHANGED_ICU_UNIT**: Binary flag (1 if patient transferred between units)

**Clinical Significance**: Different ICU types have varying complexity and typical LOS patterns. Unit transfers often indicate complications.

**Result**: Enhanced dataset with ICU unit complexity and transfer indicators.

In [ ]:
print("📊 Step 3: Creating ICU unit type features...")

base_icu_df = base_icu_df \
    .withColumn("FIRST_UNIT_ENCODED", 
                when(col("FIRST_CAREUNIT") == "MICU", 1)
                .when(col("FIRST_CAREUNIT") == "SICU", 2)
                .when(col("FIRST_CAREUNIT") == "CSRU", 3)
                .when(col("FIRST_CAREUNIT") == "CCU", 4)
                .when(col("FIRST_CAREUNIT") == "TSICU", 5)
                .otherwise(0)) \
    .withColumn("CHANGED_ICU_UNIT", 
                when(col("FIRST_CAREUNIT") != col("LAST_CAREUNIT"), 1).otherwise(0))


print("✅ Base ICU Dataset - Unit Type Features")

## Extracting Time-based Features

**Action**: Filter out invalid records where INTIME >= OUTTIME.


In [ ]:
print("📊 Step 4: Creating time-based features...")
base_icu_df = base_icu_df \
    .filter(col("ICU_INTIME") < col("ICU_OUTTIME"))
print("✅ Base ICU Dataset - Time Based Features")

## Remove Outliers (Excessive Length Of Stay)

In [ ]:
print("\n📈 ICU Length of Stay Statistics (Days):")
base_icu_df.select("ICU_LOS_DAYS").describe().show()

We kept every ICU STAY that had duration (LOS) between 0.0 and 9.1 days, considered normal legnths since:

| Statistic                | Value (days)                                    |
| ------------------------ | ----------------------------------------------- |
| **Minimum**              | 0.0 (can be admission + discharge on same day)  |
| **25th percentile (Q1)** | \~1.1                                           |
| **Median (Q2)**          | \~2.1                                           |
| **75th percentile (Q3)** | \~4.3                                           |
| **Maximum**              | \~88 (but can go slightly higher in edge cases) |
| **Mean**                 | \~3.3–3.5                                       |

Using interquartile range (IQR) method:

* IQR = Q3 - Q1 = 4.3 - 1.1 = ~3.2

* Upper Bound for outliers = Q3 + 1.5 × IQR ≈ 4.3 + 4.8 = ~9.1 days

* Lower Bound = Q1 - 1.5 × IQR ≈ 1.1 - 4.8 = < 0, which is ignored since LOS can’t be negative

So:

* Typical ICU LOS: 1.1 to 4.3 days

* Outliers: ICU stays longer than ~9.1 days

In [ ]:
print("📊 Step 5: Cleaning target variable...")

# List of desired columns
selected_columns = [
    "ICUSTAY_ID", "SUBJECT_ID", "HADM_ID", "ICU_LOS_DAYS", "ICU_INTIME", "ICU_OUTTIME", "AGE_AT_ICU_ADMISSION", "GENDER_BINARY", "CAME_FROM_ER",
    "HAS_INSURANCE", "ADMISSION_TYPE_ENCODED", "ETHNICITY_ENCODED",
    "FIRST_UNIT_ENCODED", "CHANGED_ICU_UNIT"
]

# Apply filter and select columns
base_icu_df = base_icu_df \
    .filter(col("ICU_LOS_DAYS").between(0.0, 9.1)) \
    .select(*selected_columns) \
    .cache()

print("✅ Base ICU Dataset - Remove Outliers")

print("\n📋 Sample of ICU stay records:")
base_icu_df.show(5)

## Extracting Clinical Features

**Purpose**: Extract top 20 most common CHARTEVENTS as features for ML models.

**Process**:
1. **Identify**: Find 20 most frequent CHARTEVENTS (typically vital signs)
2. **Calculate**: Average value of each test in first 24 hours of ICU stay
3. **Handle Missing**: Set missing values to **-1** (not null) for ML compatibility

**Time Window**: First 24 hours after ICU admission (INTIME + 24h)

**Result**: 20 vital signs features with consistent **-1** encoding for missing data, ensuring ML algorithm compatibility.


In [ ]:
print("📊 Identifying top 20 most frequent tests from CHARTEVENTS...")


# Get frequency count of each ITEMID in CHARTEVENTS
itemid_counts = chartevents_df \
    .filter(col("ICUSTAY_ID").isNotNull()) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("CHARTTIME").isNotNull()) \
    .groupBy("ITEMID") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(20) \
    .collect()

# Create mapping dictionary for top 20 items
top_20_items = {row["ITEMID"]: f"VITAL_{row['ITEMID']}" for row in itemid_counts}
print(f"🎯 Top 20 chart items selected: {top_20_items}")

print("📊 Filtering CHARTEVENTS for top 20 items...")

chartevents_top20 = chartevents_df \
    .filter(col("ITEMID").isin(list(top_20_items.keys()))) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("ICUSTAY_ID").isNotNull()) \
    .filter(col("CHARTTIME").isNotNull()) \
    .join(base_icu_df.select("ICUSTAY_ID", "ICU_INTIME", "ICU_OUTTIME"), "ICUSTAY_ID", "inner") \
    .filter(col("CHARTTIME").between(col("ICU_INTIME"), col("ICU_OUTTIME"))) \
    .select("ICUSTAY_ID", "ITEMID", "CHARTTIME", "VALUENUM")

# Process first 24 hours
vitals_24h_top20 = chartevents_top20.alias("ce") \
    .join(base_icu_df.select("ICUSTAY_ID", "ICU_INTIME"), "ICUSTAY_ID", "inner") \
    .filter(
        col("ce.CHARTTIME").between(
            col("ICU_INTIME"), 
            col("ICU_INTIME") + expr("INTERVAL 24 HOURS")
        )
    )

print("📊 Calculating aggregates for top 20 vitals...")

# Initialize with ICUSTAY_ID
vitals_features_top20 = base_icu_df.select("ICUSTAY_ID")

# Process each vital sign
for itemid, name in top_20_items.items():
    #print(f"Processing {name} (ITEMID={itemid})...")
    
    vital_stats = vitals_24h_top20 \
        .filter(col("ITEMID") == itemid) \
        .groupBy("ICUSTAY_ID") \
        .agg(avg("VALUENUM").alias(f"{name}_AVG"))
    
    # Left join (without filling NULLs yet)
    vitals_features_top20 = vitals_features_top20.join(vital_stats, "ICUSTAY_ID", "left")

# Fill ALL NULL values with -1 AFTER all joins are done
vitals_features_top20 = vitals_features_top20.na.fill(-1)

# Cleanup
chartevents_df.unpersist()
vitals_24h_top20.unpersist()

# Verify no NULLs remain
print(f"✅ Created {len(top_20_items)} features from top 20 vital signs (NULLs replaced with -1)")
vitals_features_top20.show(5)

## Laboratory Events

**Purpose**: Extract top 20 most common lab tests as features for ML models.

**Process**:
1. **Identify**: Find 20 most frequent LABEVENTS (blood tests, chemistry panels)
2. **Time Window**: 6 hours before ICU admission + first 24 hours in ICU (30h total)
3. **Calculate**: Average value of each lab test within the 30-hour window
4. **Handle Missing**: Set missing values to **-1** (not null) for ML compatibility

**Time Range**: ICU_INTIME - 6h to ICU_INTIME + 24h

**Result**: 20 lab test features with consistent -1 encoding for missing data, capturing pre-ICU and early ICU clinical status.

In [ ]:
print("\n🧪 Creating laboratory features from LABEVENTS...")

# Step 1: Identify top 20 most frequent lab items
print("📊 Identifying top 20 most frequent lab items...")
top_20_lab_items = labevents_df \
    .filter(col("HADM_ID").isin([row["HADM_ID"] for row in base_icu_df.select("HADM_ID").collect()])) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("VALUENUM") > 0) \
    .groupBy("ITEMID") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(20) \
    .collect()

# Create mapping dictionary (using ITEMID as name if no mapping exists)
lab_items = {row["ITEMID"]: f"LAB_{row['ITEMID']}" for row in top_20_lab_items}
print(f"🎯 Top 20 lab items selected: {list(lab_items.keys())}")

# Step 2: Filter lab events within first 24 hours of ICU stay
print("📊 Filtering LABEVENTS for top 20 items...")
labs_24h = labevents_df.alias("le") \
    .join(base_icu_df.select("ICUSTAY_ID", "HADM_ID", "ICU_INTIME"), "HADM_ID", "inner") \
    .filter(col("le.ITEMID").isin(list(lab_items.keys()))) \
    .filter(col("le.VALUENUM").isNotNull()) \
    .filter(col("le.VALUENUM") > 0) \
    .filter(
        col("le.CHARTTIME").between(
            col("ICU_INTIME") - expr("INTERVAL 6 HOURS"),  # Include pre-ICU labs
            col("ICU_INTIME") + expr("INTERVAL 24 HOURS")
        )
    )

# Step 3: Calculate lab statistics with NULL handling
print("📊 Calculating laboratory statistics...")
labs_features = base_icu_df.select("ICUSTAY_ID")

for itemid, name in lab_items.items():
    #print(f"   Processing {name} (ITEMID={itemid})...")
    
    item_stats = labs_24h \
        .filter(col("ITEMID") == itemid) \
        .groupBy("ICUSTAY_ID") \
        .agg(
            coalesce(avg("VALUENUM"), lit(-1)).alias(f"{name}_AVG")
        )
    
    labs_features = labs_features.join(item_stats, "ICUSTAY_ID", "left")

# Final NULL fill as safeguard (though coalesce should have handled it)
labs_features = labs_features.na.fill(-1)

# Cleanup
labevents_df.unpersist()
labs_24h.unpersist()

print(f"✅ Created {len(lab_items)} lab features for {labs_features.count():,} ICU stays")

# Show sample of features
print("📊 Sample features:")
labs_features.show(5, truncate=False)

## Diagnosis ICD

**Purpose**: Extract diagnosis patterns as ML features from ICD-9 codes.

**Process**:
1. **Identify**: Top 10 most frequent ICD-9 diagnosis codes
2. **Count**: Total diagnoses per admission (comorbidity burden)
3. **Create**: Binary features for each top 10 diagnosis (HAS_[CODE])

**Features Created**:
- **TOTAL_DIAGNOSES**: Count of all diagnoses (comorbidity indicator)
- **HAS_[ICD9_CODE]**: Binary flags for top 10 most common diagnoses

**Result**: 11 diagnosis features (1 count + 10 binary) capturing disease complexity and specific conditions.

In [ ]:
print("\n🏥 Creating diagnosis features from ICD codes...")

# Step 1: Identify top 10 most frequent ICD9 codes
print("📊 Identifying top 10 most frequent diagnoses...")
top_10_diagnoses = diagnoses_df \
    .groupBy("ICD9_CODE") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(10) \
    .collect()

top_10_codes = [row["ICD9_CODE"] for row in top_10_diagnoses]
print(f"🎯 Top 10 ICD9 codes: {top_10_codes}")

# import spark fn instead builtin
from pyspark.sql.functions import count as spark_count, collect_list

# Step 2: Count total diagnoses per admission (comorbidity burden)
diagnosis_features = diagnoses_df.groupBy("HADM_ID") \
    .agg(
        spark_count("ICD9_CODE").alias("TOTAL_DIAGNOSES"),
        collect_list("ICD9_CODE").alias("DIAGNOSIS_CODES")
    )

# Step 3: Create binary features for top 10 diagnoses
for code in top_10_codes:
    diagnosis_features = diagnosis_features.withColumn(
        f"HAS_{code}",
        when(array_contains(col("DIAGNOSIS_CODES"), code), 1).otherwise(0)
    )

# Drop the raw codes list
diagnosis_features = diagnosis_features.drop("DIAGNOSIS_CODES")

print(f"✅ Created {len(top_10_codes)} diagnosis features for {diagnosis_features.count():,} admissions")

# Show sample of features
print("📊 Sample features:")
diagnosis_features.show(5, truncate=False)

print(f"\n⏰ Clinical features completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Joining All Features

In [ ]:
print("📊 Joining all features...")

# Start with base ICU dataset and join all features
final_dataset = base_icu_df \
    .join(vitals_features_top20, "ICUSTAY_ID", "left") \
    .join(labs_features, "ICUSTAY_ID", "left") \
    .join(diagnosis_features, "HADM_ID", "left")

# Cleanup
base_icu_df.unpersist()
vitals_features_top20.unpersist()
labs_features.unpersist()
diagnosis_features.unpersist()

print(f"✅ All features joined! Final dataset has {final_dataset.count()} records")
print("📊 Sample of final dataset:")
final_dataset.show(5, truncate=False)

In [ ]:
print("\n📋 Step 3: Selecting final features for regression modeling...")

# Define feature columns for modeling

feature_columns = final_dataset.columns
print("\n📋 Features - Final Dataset Columns:")
print(feature_columns)


# Create modeling dataset with selected features
modeling_dataset = final_dataset.select(
    ["ICUSTAY_ID", "ICU_LOS_DAYS"] + feature_columns
)

# Remove any remaining nulls and invalid records
modeling_dataset = modeling_dataset.filter(col("ICU_LOS_DAYS").isNotNull()) \
    .filter(col("ICU_LOS_DAYS") > 0) \
    .filter(col("AGE_AT_ICU_ADMISSION").between(18,80))

# Cache the final dataset
#modeling_dataset = modeling_dataset.repartition()
modeling_dataset.cache()


print(f"✅ Final modeling dataset prepared!")
print(f"📏 Final dataset: {modeling_dataset.count():,} ICU stays")
print(f"📊 Total features: {len(feature_columns)} predictive features")
print(f"🎯 Target variable: ICU_LOS_DAYS (continuous)")


# Display sample of final dataset
#print(f"\n📋 Sample of final modeling dataset:")
#modeling_dataset.select("ICUSTAY_ID", "ICU_LOS_DAYS", "AGE_AT_ICU_ADMISSION", 
#                        "HEART_RATE_AVG", "CREATININE_FIRST", "HAS_SEPSIS").show(5)

# Basic statistics of target variable
#print(f"\n📈 Final ICU Length of Stay Statistics:")
#modeling_dataset.select("ICU_LOS_DAYS").describe().show()

print(f"\n⏰ Dataset preparation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🚀 Ready for train/test split and model training!")

## Preparing for Machine Learning

In [ ]:
print("📊 Step 1: Creating train/test split...")

# Split the data (80% train, 20% test)
train_data, test_data = modeling_dataset.randomSplit([0.8, 0.2], seed=42)

# Cache both datasets for performance
train_data.cache()
test_data.cache()


print(f"✅ Data split completed:")
#print(f"   📈 Training set: {train_data.count():,} ICU stays ({train_data.count()/modeling_dataset.count()*100:.1f}%)")
#print(f"   📊 Test set: {test_data.count():,} ICU stays ({test_data.count()/modeling_dataset.count()*100:.1f}%)")

# Show target variable distribution in both sets
#print(f"\n📈 Target variable distribution:")
#print(f"Training set LOS statistics:")
#train_data.select("ICU_LOS_DAYS").describe().show()

#print(f"Test set LOS statistics:")
#test_data.select("ICU_LOS_DAYS").describe().show()

# ============================================================================
# FEATURE VECTOR ASSEMBLY
# ============================================================================

print("\n🔧 Step 2: Assembling feature vectors...")

# Create feature vector assembler
feature_assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features_raw"
)

# Apply feature assembler to training data
train_assembled = feature_assembler.transform(train_data)
test_assembled = feature_assembler.transform(test_data)


train_data.unpersist()
test_data.unpersist()


print(f"✅ Feature vectors assembled:")
print(f"   📊 Feature vector size: {len(feature_columns)} dimensions")

# ============================================================================
# FEATURE SCALING
# ============================================================================

print("\n⚖️ Step 3: Scaling features...")

# Create StandardScaler to normalize features
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)

# Fit scaler on training data
scaler_model = scaler.fit(train_assembled)
train_scaled = scaler_model.transform(train_assembled)
test_scaled = scaler_model.transform(test_assembled)

# Cache the final processed datasets
train_scaled.cache()
test_scaled.cache()


print(f"✅ Feature scaling completed:")
print(f"   📊 Features standardized (mean=0, std=1)")
print(f"   🔧 Scaler fitted on training data only")


## Final Dataset Preparation

In [ ]:

print("\n📋 Step 4: Preparing final ML datasets...")

# Select columns needed for modeling
ml_columns = ["ICUSTAY_ID", "ICU_LOS_DAYS", "features"]

train_final = train_scaled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")
test_final = test_scaled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")

#train_final = train_assembled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")
#test_final = test_assembled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")


# train_final = train_assembled.select("ICUSTAY_ID", "ICU_LOS_DAYS", "features_raw") \
#     .withColumnRenamed("ICU_LOS_DAYS", "label") \
#     .withColumnRenamed("features_raw", "features")

# test_final = test_assembled.select("ICUSTAY_ID", "ICU_LOS_DAYS", "features_raw") \
#     .withColumnRenamed("ICU_LOS_DAYS", "label") \
#     .withColumnRenamed("features_raw", "features")




print("\n📋 Caching...")

# Cache final datasets
train_final.cache()
test_final.cache()

print(f"✅ Final ML datasets prepared:")
print(f"   🎯 Target variable: 'label' (ICU_LOS_DAYS)")
print(f"   📊 Features: 'features' (scaled vector)")
print(f"   🔑 Identifier: 'ICUSTAY_ID'")

# Show sample of final datasets
print(f"\n📋 Sample of training data structure:")
train_final.select("ICUSTAY_ID", "label").show(3)

print(f"\n📋 Feature vector example (first 10 features):")
# Show first few elements of feature vector for one sample
sample_features = train_final.select("features").take(1)[0]["features"]
print(f"   📊 Feature vector sample: {sample_features.toArray()[:10]}...")
print(f"   📏 Total feature dimensions: {len(sample_features.toArray())}")

# ============================================================================
# DATA QUALITY CHECKS
# ============================================================================

print(f"\n🔍 Step 5: Final data quality checks...")

# Check for any remaining nulls
train_nulls = train_final.filter(col("label").isNull() | col("features").isNull()).count()
test_nulls = test_final.filter(col("label").isNull() | col("features").isNull()).count()

print(f"   🔍 Null values in training set: {train_nulls}")
print(f"   🔍 Null values in test set: {test_nulls}")

# Show target variable ranges
# train_stats = train_final.agg(
#     min("label").alias("min_los"),
#     max("label").alias("max_los"), 
#     avg("label").alias("mean_los"),
#     stddev("label").alias("std_los")
# ).collect()[0]

# print(f"\n📊 Final training set target statistics:")
# print(f"   📉 Min LOS: {train_stats['min_los']:.2f} days")
# print(f"   📈 Max LOS: {train_stats['max_los']:.2f} days") 
# print(f"   📊 Mean LOS: {train_stats['mean_los']:.2f} days")
# print(f"   📏 Std LOS: {train_stats['std_los']:.2f} days")

print(f"\n✅ Data preprocessing completed successfully!")
print(f"🚀 Ready for model training with {len(feature_columns)} features")


print(f"⏰ Preprocessing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Training Multiple Models

In [ ]:
print("📊 Step 1: Setting up evaluation metrics...")

# Create regression evaluators
rmse_evaluator = RegressionEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="rmse"
)

mae_evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction", 
    metricName="mae"
)

r2_evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="r2"
)

print("✅ Evaluation metrics configured: RMSE, MAE, R²")

### Linear Regression

In [ ]:
print("\n📈 Step 2: Training Linear Regression model...")
print(f"🕐 Started at: {datetime.now().strftime('%H:%M:%S')}")
start_time = time.time()

# Create Linear Regression model
lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=200,                    # Increased for better convergence
    regParam=0.001,                 # Lower regularization for healthcare data
    elasticNetParam=0.1,            # Slight L1 penalty for feature selection
    tol=1e-8,                       # Tighter tolerance for precision
    standardization=False,          # We're doing manual scaling
    fitIntercept=True,
    aggregationDepth=3,             # Better for distributed training
    loss="squaredError",
    solver="normal"                 # Best for small-medium datasets
)


# Train the model
print("   🔄 Training Linear Regression...")
lr_model = lr.fit(train_final)

print("   🔄 Linear Regression - Making predictions (test data)...")
lr_predictions = lr_model.transform(test_final)

print("   🔄 Linear Regression - Evaluation...")
lr_rmse = rmse_evaluator.evaluate(lr_predictions)
lr_mae = mae_evaluator.evaluate(lr_predictions)
lr_r2 = r2_evaluator.evaluate(lr_predictions)

print(f"✅ Linear Regression Results:")
print(f"   📉 RMSE: {lr_rmse:.3f} days")
print(f"   📊 MAE: {lr_mae:.3f} days")
print(f"   📈 R²: {lr_r2:.3f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"🕐 Completed at: {datetime.now().strftime('%H:%M:%S')}")
print(f"⏱️ Total elapsed time: {elapsed_time:.2f} seconds")


# Linear Regression Predictions

print("\n📈 Linear Regression Predictions (Sample 20):")
lr_display = lr_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

lr_display.show(20, truncate=False)

### Random Forest

In [ ]:

print("\n🌲 Step 3: Training Random Forest model...")
print(f"🕐 Started at: {datetime.now().strftime('%H:%M:%S')}")
start_time = time.time()

# Create Random Forest model
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="label",
    numTrees=200,                   # More trees = better accuracy (if enough cores/memory)
    maxDepth=12,                    # Deeper trees capture more complexity
    minInstancesPerNode=2,          # Allows more granular splits
    subsamplingRate=0.9,            # Slightly higher sample rate for stability
    featureSubsetStrategy="sqrt",   # Good default for regression
    seed=42                         # Reproducibility
)

print("   🔄 Training Random Forest...")
rf_model = rf.fit(train_final)

print("   🔄 Random Forest - Making predictions (test data)...")
rf_predictions = rf_model.transform(test_final)

print("   🔄 Random Forest - Evaluation...")
rf_rmse = rmse_evaluator.evaluate(rf_predictions)
rf_mae = mae_evaluator.evaluate(rf_predictions)
rf_r2 = r2_evaluator.evaluate(rf_predictions)

print(f"✅ Random Forest Results:")
print(f"   📉 RMSE: {rf_rmse:.3f} days")
print(f"   📊 MAE: {rf_mae:.3f} days")
print(f"   📈 R²: {rf_r2:.3f}")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"🕐 Completed at: {datetime.now().strftime('%H:%M:%S')}")
print(f"⏱️ Total elapsed time: {elapsed_time:.2f} seconds")


# Random Forest Predictions
print("\n🌲 Random Forest Predictions (Sample 20):")
rf_display = rf_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

rf_display.show(20, truncate=False)


## Model Comparison

In [ ]:
print("\n🏆 Step 5: Model Performance Comparison...")

# Create comparison summary
results_data = [
    ("Linear Regression", lr_rmse, lr_mae, lr_r2),
    ("Random Forest", rf_rmse, rf_mae, rf_r2)
]

results_df = spark.createDataFrame(results_data, ["Model", "RMSE", "MAE", "R2"])

print("📊 Model Performance Summary:")
results_df.show(truncate=False)

# Find best model
import operator
import builtins
best_rmse_model = builtins.min(results_data, key=operator.itemgetter(1))
best_r2_model = builtins.max(results_data, key=operator.itemgetter(3))

print(f"\n🥇 Best Models:")
print(f"   🎯 Lowest RMSE: {best_rmse_model[0]} ({best_rmse_model[1]:.3f} days)")
print(f"   📈 Highest R²: {best_r2_model[0]} ({best_r2_model[3]:.3f})")

## Display Predictions

In [ ]:

# Linear Regression Predictions

print("\n📈 Linear Regression Predictions (Sample 20):")
lr_display = lr_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

lr_display.show(20, truncate=False)



# Random Forest Predictions
print("\n🌲 Random Forest Predictions (Sample 20):")
rf_display = rf_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

rf_display.show(20, truncate=False)